In [ ]:
import pandas as pd
df = pd.read_csv( './drive/MyDrive/Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )
df

In [ ]:
!pip install sentencepiece transformers


In [20]:
df_test_= pd.read_csv( './drive/MyDrive/Data/pcl_test.tsv', sep = '\t', names=['seq','id','info','country', 'text','label_'] )
df_test_

,seq,id,info,country,text,label_
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to...",NaN
1,t_1,@@16397324,women,pk,In most poor households with no education chil...,NaN
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...,NaN
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population...",NaN
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C...",NaN
...,...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...,NaN
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...,NaN
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina...",NaN
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...,NaN


In [ ]:
df_test= df_test_[['text']]
df_test

In [ ]:
df_test['label_'] = 1
df_test

In [ ]:
import torch

from transformers import RobertaTokenizer, RobertaModel
import pandas as pd
df = pd.read_csv('./drive/MyDrive/Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )

df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

# Importing the libraries needed
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
df_final = df[['text','label']]
#df_data = data.dropna(subset=['id','info','country'])
df_final                  

df_final.shape
#df_final.columns
df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
df_final
df_final.drop('label', axis = 1, inplace= True)
print(df_final)

nclasses = len(list(df_final.label_.unique()))
nclasses

my_classes = {c:i for i, c in enumerate(list(df_final.label_.unique()))}
df_final['label_'] = [my_classes[l] for l in df_final.label_]

MAX_LEN= 512
TRAIN_BS = 8
VALID_BS = 8
EPOCHS = 1
LR =  1e-05
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

class PclData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len); print(self.data)
        
    def __getitem__(self, index):# function compulsory
        sent = self.data.text[index]
       # sent = " ".join(sent.split())
        inputs = self.tokenizer.encode_plus(
        sent, 
        None,
        add_special_tokens = True,
        max_length=self.max_len,
        padding = 'max_length',
        return_token_type_ids = True,
        truncation=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return { 'ids': torch.tensor(ids, dtype=torch.long),
               'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.data.label_[index], dtype=torch.long)
                
               }
    
    def __len__(self):
        return self.len
        

In [ ]:
train_size = 0.8
train_dataset = df_final.sample(frac= train_size, random_state=20)
val_dataset = df_final.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = df_test

#train_dataset_1 = [ 0 if (y == 1 or y == 0) else 1 for y in train_dataset ]
#test_dataset_1 = [ 0 if (y == 1 or y == 0) else 1 for y in test_dataset ]


print("FULL Dataset: {}".format(df_final.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = PclData(train_dataset, tokenizer, MAX_LEN)
val_set = PclData(val_dataset, tokenizer, MAX_LEN)
test_set= PclData(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BS,
               'shuffle':True,
               'num_workers': 2}
val_params = {'batch_size': VALID_BS,
              'shuffle': False,
              'num_workers': 2}

trainloader = DataLoader(training_set, **train_params)
valloader = DataLoader(val_set, **val_params)
testloader = DataLoader(test_set, **val_params)

In [11]:
class RobertaModelClass (torch.nn.Module):
    def __init__(self, nclasses):
        super(RobertaModelClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained('roberta-base')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, nclasses)

    def forward(self, input_ids, attention_mask):
        with torch.autograd.no_grad():
            output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaModelClass(nclasses)
model.to(device)

In [13]:
import numpy as np
wt_array =len(df_final['text'])/(len(set(df_final['label_']))*(np.bincount(df_final['label_'])))
wt_array

array([0.55240106, 5.27089627])

In [14]:
class_weights=torch.FloatTensor(wt_array).cuda()

In [15]:
loss_function = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(params =model.parameters(), lr=LR)
def calculate_acc(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct


In [16]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/textclassify_experiment_1')


In [17]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.train()
    for _, data in tqdm(enumerate(trainloader, 0)):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)
        
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_acc(big_idx, targets)
        
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if _ % 50 == 0:
            
            loss_step = tr_loss/nb_tr_steps
            acc_step = (n_correct*100)/nb_tr_examples
            print(f'Training Loss per 50 steps: {loss_step}, Training Accuracy: {acc_step}')
            writer.add_scalar('training_loss', loss_step, epoch*len(trainloader) +_)
            
            
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    print(f'Total Accuracy Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Training Loss Epoch: {epoch_loss}, Training Accuracy Epoch: {epoch_acc}')
    valid(model, valloader)
    return

In [18]:
# Validation

def valid(model, testloader):
    model.eval()
    n_correct =0; n_wrong =0; total=0; tr_loss =0; nb_tr_steps =0; nb_tr_examples=0
    y_pred, y_true = [],[]
    with torch.no_grad():
        for _, data in enumerate(testloader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_acc(big_idx, targets)

            y_true.extend(targets.cpu().detach().numpy())
      
            y_pred.extend(torch.argmax(outputs, dim=1).cpu().detach().numpy())
            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)
            
            if _ % 100 == 0:
                loss_step = tr_loss/nb_tr_steps
                acc_step = (n_correct*100)/nb_tr_examples
                print(f'Validation Loss per 100 steps: {loss_step}, Validation Accuracy per: {acc_step}')   
        
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Validation Loss Epoch: {epoch_loss}, Validation Accuracy Epoch: {epoch_acc}')
    
    return epoch_acc, y_true, y_pred

In [19]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 50 steps: 0.7471062541007996, Training Accuracy: 12.5
Validation Loss per 100 steps: 0.7198158502578735, Validation Accuracy per: 0.0
Validation Loss per 100 steps: 0.698964268264204, Validation Accuracy per: 10.891089108910892
Validation Loss per 100 steps: 0.6997426780895214, Validation Accuracy per: 10.572139303482587


1it [02:21, 141.63s/it]

Validation Loss Epoch: 0.7002603311575096, Validation Accuracy Epoch: 10.02865329512894


50it [02:47,  1.88it/s]

...

Training Loss per 50 steps: 0.6514814058979312, Training Accuracy: 89.1983016983017
Validation Loss per 100 steps: 0.44115304946899414, Validation Accuracy per: 100.0
Validation Loss per 100 steps: 0.6692290645424682, Validation Accuracy per: 89.10891089108911
Validation Loss per 100 steps: 0.6580378036593917, Validation Accuracy per: 89.42786069651741


1001it [57:07, 41.70s/it]

Validation Loss Epoch: 0.651176254940397, Validation Accuracy Epoch: 89.97134670487107


1047it [57:32,  3.30s/it]

Total Accuracy Epoch 0: 89.26438977788392
Training Loss Epoch: 0.6507179651412945, Training Accuracy Epoch: 89.26438977788392


In [24]:
acc, y_true, y_pred = valid(model, testloader)

Validation Loss per 100 steps: 1.0523362159729004, Validation Accuracy per: 0.0
Validation Loss per 100 steps: 1.0477707220776247, Validation Accuracy per: 0.0
Validation Loss per 100 steps: 1.0483255902333046, Validation Accuracy per: 0.0
Validation Loss per 100 steps: 1.048447054485942, Validation Accuracy per: 0.0
Validation Loss per 100 steps: 1.0483714767227743, Validation Accuracy per: 0.0
Validation Loss Epoch: 1.0483138683195652, Validation Accuracy Epoch: 0.0


In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
accuracy = confusion_matrix(y_true, y_pred)
print(accuracy)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

[[   0    0]
 [3832    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00    3832.0

    accuracy                           0.00    3832.0
   macro avg       0.00      0.00      0.00    3832.0
weighted avg       0.00      0.00      0.00    3832.0



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
y_true

In [28]:
f =open('bert_roberta_small_test_1.txt', 'w')
for i in y_pred:
  print(i, file = f )
f.close()
